In [2]:
import torch
import torchvision
import syft as sy
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [3]:
data_dir = './data/cifar10'

In [4]:
hook = sy.TorchHook(torch)

In [5]:
jake = sy.VirtualWorker(hook, id="jake")
john = sy.VirtualWorker(hook, id="john")

In [6]:
train_dataset = ImageFolder(data_dir + '/train', transform = ToTensor())
test_dataset = ImageFolder(data_dir + '/test', transform = ToTensor())

In [7]:
federated_train_loader = sy.FederatedDataLoader(
    train_dataset.federate((jake, john)), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True)

In [8]:
def show_batch(dl):
    for images, labels in dl:
        print(images.size())
        print(labels.size())
        break

In [9]:
show_batch(test_loader)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [11]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    'epochs' : 10
}

In [22]:
# create a simple CNN net
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # output bs*32*16*16
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2) # output bs*64*2*2
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=576, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )

        self.dropout = nn.Dropout2d(0.25)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 576)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [13]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()

    # iterate over federated data
    for batch_idx, (data, target) in enumerate(train_loader):

        # send the model to the remote location 
        model = model.send(data.location)

        # the same torch code that we are use to
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        # this loss is a ptr to the tensor loss 
        # at the remote location
        loss = F.nll_loss(output, target)

        # call backward() on the loss ptr,
        # that will send the command to call
        # backward on the actual loss tensor
        # present on the remote machine
        loss.backward()

        optimizer.step()

        # get back the updated model
        model.get()

        if batch_idx % args['log_interval'] == 0:

            # a thing to note is the variable loss was
            # also created at remote worker, so we need to
            # explicitly get it back
            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader), 
                    loss.item()
                )
            )

In [14]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=args['lr'])

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/50048 (0%)]	Loss: 2.304238
Train Epoch: 1 [640/50048 (1%)]	Loss: 2.250695
Train Epoch: 1 [1280/50048 (3%)]	Loss: 2.210575
Train Epoch: 1 [1920/50048 (4%)]	Loss: 2.157618
Train Epoch: 1 [2560/50048 (5%)]	Loss: 2.097458
Train Epoch: 1 [3200/50048 (6%)]	Loss: 2.014193
Train Epoch: 1 [3840/50048 (8%)]	Loss: 1.903906
Train Epoch: 1 [4480/50048 (9%)]	Loss: 1.790621
Train Epoch: 1 [5120/50048 (10%)]	Loss: 1.699381
Train Epoch: 1 [5760/50048 (12%)]	Loss: 1.668174
Train Epoch: 1 [6400/50048 (13%)]	Loss: 1.640497
Train Epoch: 1 [7040/50048 (14%)]	Loss: 1.620039
Train Epoch: 1 [7680/50048 (15%)]	Loss: 1.621258
Train Epoch: 1 [8320/50048 (17%)]	Loss: 1.632844
Train Epoch: 1 [8960/50048 (18%)]	Loss: 1.612172
Train Epoch: 1 [9600/50048 (19%)]	Loss: 1.626764
Train Epoch: 1 [10240/50048 (20%)]	Loss: 1.602517
Train Epoch: 1 [10880/50048 (22%)]	Loss: 1.606276
Train Epoch: 1 [11520/50048 (23%)]	Loss: 1.624046
Train Epoch: 1 [12160/50048 (24%)]	Loss: 1.611804
Train Epoch: 1 [12800/50048 

Train Epoch: 3 [3200/50048 (6%)]	Loss: 1.394798
Train Epoch: 3 [3840/50048 (8%)]	Loss: 1.520364
Train Epoch: 3 [4480/50048 (9%)]	Loss: 1.551208
Train Epoch: 3 [5120/50048 (10%)]	Loss: 1.353395
Train Epoch: 3 [5760/50048 (12%)]	Loss: 1.410080
Train Epoch: 3 [6400/50048 (13%)]	Loss: 1.439280
Train Epoch: 3 [7040/50048 (14%)]	Loss: 1.365974
Train Epoch: 3 [7680/50048 (15%)]	Loss: 1.405336
Train Epoch: 3 [8320/50048 (17%)]	Loss: 1.416731
Train Epoch: 3 [8960/50048 (18%)]	Loss: 1.295581
Train Epoch: 3 [9600/50048 (19%)]	Loss: 1.426612
Train Epoch: 3 [10240/50048 (20%)]	Loss: 1.327028
Train Epoch: 3 [10880/50048 (22%)]	Loss: 1.553579
Train Epoch: 3 [11520/50048 (23%)]	Loss: 1.457275
Train Epoch: 3 [12160/50048 (24%)]	Loss: 1.487447
Train Epoch: 3 [12800/50048 (26%)]	Loss: 1.392838
Train Epoch: 3 [13440/50048 (27%)]	Loss: 1.276032
Train Epoch: 3 [14080/50048 (28%)]	Loss: 1.387896
Train Epoch: 3 [14720/50048 (29%)]	Loss: 1.339422
Train Epoch: 3 [15360/50048 (31%)]	Loss: 1.464117
Train Epoch: 3

Train Epoch: 5 [6400/50048 (13%)]	Loss: 1.277691
Train Epoch: 5 [7040/50048 (14%)]	Loss: 1.119711
Train Epoch: 5 [7680/50048 (15%)]	Loss: 1.317215
Train Epoch: 5 [8320/50048 (17%)]	Loss: 1.253503
Train Epoch: 5 [8960/50048 (18%)]	Loss: 1.501366
Train Epoch: 5 [9600/50048 (19%)]	Loss: 1.174926
Train Epoch: 5 [10240/50048 (20%)]	Loss: 1.251156
Train Epoch: 5 [10880/50048 (22%)]	Loss: 1.292401
Train Epoch: 5 [11520/50048 (23%)]	Loss: 1.294362
Train Epoch: 5 [12160/50048 (24%)]	Loss: 1.139868
Train Epoch: 5 [12800/50048 (26%)]	Loss: 1.162836
Train Epoch: 5 [13440/50048 (27%)]	Loss: 1.218704
Train Epoch: 5 [14080/50048 (28%)]	Loss: 1.022158
Train Epoch: 5 [14720/50048 (29%)]	Loss: 1.164927
Train Epoch: 5 [15360/50048 (31%)]	Loss: 1.236923
Train Epoch: 5 [16000/50048 (32%)]	Loss: 1.005167
Train Epoch: 5 [16640/50048 (33%)]	Loss: 1.138367
Train Epoch: 5 [17280/50048 (35%)]	Loss: 1.187400
Train Epoch: 5 [17920/50048 (36%)]	Loss: 1.226922
Train Epoch: 5 [18560/50048 (37%)]	Loss: 1.288286
Train 

Train Epoch: 7 [9600/50048 (19%)]	Loss: 1.164559
Train Epoch: 7 [10240/50048 (20%)]	Loss: 0.923925
Train Epoch: 7 [10880/50048 (22%)]	Loss: 0.882428
Train Epoch: 7 [11520/50048 (23%)]	Loss: 1.062252
Train Epoch: 7 [12160/50048 (24%)]	Loss: 0.724704
Train Epoch: 7 [12800/50048 (26%)]	Loss: 0.962698
Train Epoch: 7 [13440/50048 (27%)]	Loss: 1.036913
Train Epoch: 7 [14080/50048 (28%)]	Loss: 1.017223
Train Epoch: 7 [14720/50048 (29%)]	Loss: 0.883209
Train Epoch: 7 [15360/50048 (31%)]	Loss: 1.070466
Train Epoch: 7 [16000/50048 (32%)]	Loss: 0.935717
Train Epoch: 7 [16640/50048 (33%)]	Loss: 1.224811
Train Epoch: 7 [17280/50048 (35%)]	Loss: 1.102363
Train Epoch: 7 [17920/50048 (36%)]	Loss: 1.117949
Train Epoch: 7 [18560/50048 (37%)]	Loss: 0.959182
Train Epoch: 7 [19200/50048 (38%)]	Loss: 1.107941
Train Epoch: 7 [19840/50048 (40%)]	Loss: 0.972160
Train Epoch: 7 [20480/50048 (41%)]	Loss: 0.997495
Train Epoch: 7 [21120/50048 (42%)]	Loss: 1.077318
Train Epoch: 7 [21760/50048 (43%)]	Loss: 0.996902
T

Train Epoch: 9 [12800/50048 (26%)]	Loss: 1.055329
Train Epoch: 9 [13440/50048 (27%)]	Loss: 0.862610
Train Epoch: 9 [14080/50048 (28%)]	Loss: 0.956215
Train Epoch: 9 [14720/50048 (29%)]	Loss: 1.052734
Train Epoch: 9 [15360/50048 (31%)]	Loss: 0.901689
Train Epoch: 9 [16000/50048 (32%)]	Loss: 0.869646
Train Epoch: 9 [16640/50048 (33%)]	Loss: 1.046412
Train Epoch: 9 [17280/50048 (35%)]	Loss: 0.819834
Train Epoch: 9 [17920/50048 (36%)]	Loss: 1.090525
Train Epoch: 9 [18560/50048 (37%)]	Loss: 0.954119
Train Epoch: 9 [19200/50048 (38%)]	Loss: 0.968223
Train Epoch: 9 [19840/50048 (40%)]	Loss: 0.815727
Train Epoch: 9 [20480/50048 (41%)]	Loss: 1.004352
Train Epoch: 9 [21120/50048 (42%)]	Loss: 0.874320
Train Epoch: 9 [21760/50048 (43%)]	Loss: 0.884369
Train Epoch: 9 [22400/50048 (45%)]	Loss: 0.894453
Train Epoch: 9 [23040/50048 (46%)]	Loss: 0.971830
Train Epoch: 9 [23680/50048 (47%)]	Loss: 0.878662
Train Epoch: 9 [24320/50048 (49%)]	Loss: 0.978615
Train Epoch: 9 [24960/50048 (50%)]	Loss: 0.862984


In [24]:
torch.save(model.state_dict(), 'CIFER10CNN.pth')